# Введение в TensorFlow

Принципы работы с tensorflow достаточно просты. Мы должны составить граф операций, затем передать в этот граф данные и дать команду произвести вычисления.

## Реализация нейрона

### Graph mode (v1.x)

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


Для начала определим входы графа.

In [2]:
x = tf.keras.Input(shape=(2,))
y = tf.keras.Input(shape=(1,))

tf.Variable - обучаемые переменные. 

`tf.Variable(
    initial_value=None, trainable=None, validate_shape=True, caching_device=None,
    name=None, variable_def=None, dtype=None, import_scope=None, constraint=None,
    synchronization=tf.VariableSynchronization.AUTO,
    aggregation=tf.compat.v1.VariableAggregation.NONE, shape=None
)`

In [3]:
weights = tf.Variable(shape = (1, 2), initial_value=((0.1, 0.1),), dtype=tf.float32, name="w")
bias = tf.Variable(initial_value=0.0, dtype=tf.float32, name="b")

Присвоим переменной activation функцию `tf.keras.activations.sigmoid` - сигмоида.

In [4]:
activation = tf.keras.activations.sigmoid

Свяжем узлы в граф с помощью функций + и -. Для умножения матрицы на вектор используем функцию `tf.matmul`.

In [5]:
model = activation(tf.matmul(x, tf.transpose(weights)) + bias)

Любые вычисления в TensorFlow v1.x выполняются с помощью сессии. Создадим сессию с помощью команды `tf.Session()`.

In [6]:
session = tf.Session()

Перед началом работы необходимо инициализировать переменные.

In [7]:
tf.global_variables_initializer().run(session=session)

Попробуем подать модели что-нибудь на вход. Для этого воспользуемся методом `session.run`. Первый аргумент - список с графами, выходы которых необходимо вычислить, второй - словарь с парами вход графа - данные. Метод возвращает значения, полученные на выходе каждого графа из первого аргумента соответственно.

In [8]:
output = session.run([model], feed_dict={x:np.array([[1.0, 1.0]])})
print(output)

[array([[0.54983395]], dtype=float32)]


Попробуем обучить модель функции "И". 

In [9]:
x_train=np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]]).astype('float32')
y_train=np.array([0, 0, 0, 1]).astype('float32').reshape((4, 1))

Определим loss-функцию - тоже граф.

In [10]:
loss = tf.reduce_mean(tf.square(model-y))

Создадим оптимизатор и настроим его на минимизацию нашей loss-функции.

In [11]:
optimizer = tf.train.RMSPropOptimizer(0.001)
optimizer_opt = optimizer.minimize(loss)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


После создания оптимизатора придется ещё раз инициализировать переменные.

In [12]:
tf.global_variables_initializer().run(session=session)

Просто несколько раз подадим оптимизатор в `sess.run`.

In [13]:
epochs = 5000
feed_dict={x:x_train, y:y_train}

for epoch in range(epochs):
    _, l = session.run([optimizer_opt, loss], feed_dict=feed_dict)
    if epoch % 100 == 0:
        print('.', end = '')

..................................................

In [14]:
output = session.run([model], feed_dict={x:np.array([[1.1, 1.1], [-0.1, -0.1]])})
print(output)

[array([[0.90961134],
       [0.0036457 ]], dtype=float32)]


После работы сессию необходимо закрыть.

In [15]:
session.close()

Для удобства можно использовать конструкцию `with`, после выхода из которой сессия закроется сама. Кроме того, больше нет необходимости передавать сессию как аргумент в некоторые функции и методы.

In [16]:
with tf.Session() as session:
    tf.global_variables_initializer().run()
    feed_dict={x:x_train, y:y_train}
    
    epochs = 5000
    for epoch in range(epochs):
        _, l = session.run([optimizer_opt, loss], feed_dict=feed_dict)
        if epoch % 100 == 0:
            print('.', end = '')
    
    print()
    output = session.run([model], feed_dict={x:np.array([[1.1, 1.1], [-0.1, -0.1]])})
    print(output)

..................................................
[array([[0.90961134],
       [0.0036457 ]], dtype=float32)]


### Eager mode (v2.x)

Основная идея api v2 - перевести графы в функции python. API v1: определение графа, вызов `sesssion.run`. API v2: определение функций python, вызов определенных функций.

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.1.0


tf.Variable - обучаемые переменные. 

`tf.Variable(
    initial_value=None, trainable=None, validate_shape=True, caching_device=None,
    name=None, variable_def=None, dtype=None, import_scope=None, constraint=None,
    synchronization=tf.VariableSynchronization.AUTO,
    aggregation=tf.compat.v1.VariableAggregation.NONE, shape=None
)`

In [2]:
weights = tf.Variable(shape = (1, 2), initial_value=((0.1, 0.1),), dtype=tf.float32, name="w")
bias = tf.Variable(initial_value=0.0, dtype=tf.float32, name="b")

Присвоим переменной activation функцию `tf.keras.activations.sigmoid` - сигмоида.

In [3]:
activation = tf.keras.activations.sigmoid

Свяжем узлы в граф с помощью функций + и -. Для умножения матрицы на вектор используем функцию `tf.matmul`. Для определения модели объявим функцию и применим [декоратор](https://pythonworld.ru/osnovy/dekoratory.html) `@tf.function`

In [4]:
@tf.function
def model(x):
    return activation(tf.add(tf.matmul(x, tf.transpose(weights)), bias))

Попробуем что-нибудь подать в такую модель.

In [5]:
output = model(tf.constant([[1.0, 1.0], [0.0, 0.0]]))
print(output)

tf.Tensor(
[[0.54983395]
 [0.5       ]], shape=(2, 1), dtype=float32)


Попробуем обучить модель функции "И". 

In [6]:
x_train=np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]]).astype('float32')
y_train=np.array([0, 0, 0, 1]).astype('float32').reshape((4, 1))

Для этого определим лосс-функцию - в v2 это должна быть тоже функция python. В данном случае можно использовать инструкцию `lambda` для создания [анонимной функции](https://pythonworld.ru/tipy-dannyx-v-python/vse-o-funkciyax-i-ix-argumentax.html) без аргументов и с возвращаемым значением, которое необходимо минимизировать.

In [8]:
loss = lambda: tf.reduce_mean(tf.square(tf.subtract(model(x_train), y_train)))

Соберем параметры, которые необходимо обучить.

In [9]:
var_list = [weights, bias]

Также потребуется определить оптимизатор.

In [10]:
epochs = 5000
optimizer = tf.keras.optimizers.RMSprop(0.001)

Теперь можно запустить процесс обучения.

In [11]:
for epoch in range(epochs):
    optimizer.minimize(loss, var_list)
    if epoch % 100 == 0:
        print('.', end = '')

..................................................

In [12]:
output = model(tf.constant([[1.1, 1.1]]))
print(output)
output = model(tf.constant([[-0.1, -0.1]]))
print(output)

tf.Tensor([[0.91108435]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.0035015]], shape=(1, 1), dtype=float32)


## Сверточная нейронная сеть для классификации MNIST

### Graph mode (v1.x)

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from tensorflow.compat.v1 import keras

import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


Загрузка данных.

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test  = x_test .astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test  = np.reshape(x_test,  (len(x_test),  28, 28, 1))

y_train_cat = keras.utils.to_categorical(y_train)
y_test_cat = keras.utils.to_categorical(y_test)

Для создания батчей "на месте" объявим генератор. 

*Пользоваться python для подобных задач - медленно, однако аналогов пока не нашел*.

In [3]:
def datagen(x, y, batch_size, epochs, shuffle=True):
    for epoch in range(epochs):
        if shuffle:
            p = np.random.permutation(len(x))
            x, y = x[p], y[p]
        for i in range(len(x)//batch_size):
            yield x[i*batch_size:(i+1)*batch_size], y[i*batch_size:(i+1)*batch_size]

Создание нейронной сети с помощью keras. Один из способов потом получить переменные сети - `tf.variable_scope`.

In [4]:
height = 28
width = 28
channels = 1

In [5]:
with tf.variable_scope('mnist_classifier'):
    model_input = keras.layers.Input(shape=(height, width, channels))
    x = keras.layers.Conv2D(32, 3)(model_input)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Conv2D(64, 4, strides=2)(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = keras.layers.Dropout(0.25)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(128, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)

    # Classification layer
    model_output = keras.layers.Dense(10, activation='softmax')(x)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Пользуясь синтаксисом TensorFlow, объявим граф вычислений для целевой функции.

In [6]:
y = keras.layers.Input(shape=(10, ))
loss = tf.losses.softmax_cross_entropy(y, model_output)

Чтобы получить переменные сети из коллекции, созданной выше, воспользуемся следующей командой.

In [7]:
train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "mnist_classifier")

Ещё один способ получить параметры: воспользоваться `keras.Model`. **Важно**: могут возникнуть проблемы с `Lambda`-слоями.

In [7]:
model = keras.Model(model_input, model_output)
train_vars = model.trainable_weights

Вместе с loss-функцией в оптимизатор можно передать список с переменными, которые необходимо обучать. Это полезно в тех случаях, когда в одном графе вычислений присутствует несколько сетей, а обучать необходимо только одну.

In [8]:
optimizer = tf.train.AdamOptimizer(0.001)
step_opt = optimizer.minimize(loss, var_list=train_vars)

Запустим алгоритм обучения.

In [9]:
epochs = 5
batch_size = 128
buffer_size = 1000

dg = datagen(x_train, y_train_cat, batch_size, epochs)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

count = 0
for (x_batch, y_batch) in dg:
    _, l = sess.run([step_opt, loss], feed_dict={model_input:x_batch, y:y_batch})
    count += 1
    
    if count % (len(x_train)//batch_size//10) == 0:
        print('.', end='')
    if count >= len(x_train)//batch_size:
        count = 0
        print(l)

..........1.4964266
..........1.469913
..........1.483226
..........1.4807158
..........1.4764936


Посчитаем точность полученной модели с помощью `tf.metrics.accuracy`. **Обратите внимание**: для работы после объявления графа для точности необходимо инициализировать локальные переменные с помощью `tf.local_variables_initializer()`.

In [10]:
acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(y, 1), 
                                  predictions=tf.argmax(model_output, 1))
sess.run(tf.local_variables_initializer())

accuracy = sess.run([acc_op], feed_dict={model_input:x_test, y:y_test_cat})
print(accuracy)

[0.9832]


Закроем сессию.

In [11]:
sess.close()

## Источники

[Документация. Работа с оптимизаторами.](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Optimizer)

[Сравнение синтаксиса v1.x и v2.x](https://github.com/tensorflow/community/blob/master/rfcs/20180918-functions-not-sessions-20.md)

[Пример TensorFlow v2 для MNIST](https://github.com/dragen1860/TensorFlow-2.x-Tutorials/tree/master/03-Play-with-MNIST)